In [45]:
import pandas as pd, numpy as np, time
pd.set_option('display.max_columns', None)

In [46]:
df = pd.read_csv("Arduino_20171220_125224.csv")

In [47]:
df.rename(columns = {'Time UTC (YYYYMMDD_HHMMSS.mmm)':'time_glob','Time Arduino (Micros since start)':'time_ard'}, 
          inplace = True)

In [48]:
df["time_glob"][1]

'20171220_175225.837'

In [49]:
# Extracting dates in the form of substrings
df["time_global"] = df["time_glob"].str[:4] + '-' + df["time_glob"].str[4:6] + '-' + df["time_glob"].str[6:8] + ' ' + df["time_glob"].str[9:11] + ':' + df["time_glob"].str[11:13] + ':' + df["time_glob"].str[13:15] + ':' + df["time_glob"].str[16:] 

In [50]:
df["date"] = pd.to_datetime(df["time_global"], format = '%Y-%m-%d %H:%M:%S:%f')

In [51]:
# Converting Arduino time to milliseconds
df["time_ard_mill"] = df["time_ard"]/1000
df.drop(columns = ['time_glob', 'time_global', 'time_ard'], axis = 1, inplace = True)

In [52]:
# Subsetting dataset
df = df[['date', 'time_ard_mill', 'Seat1', 'Seat2', 'Seat3', 'Seat4', 'Seat5', 'Seat6',
        'SeatBack1', 'SeatBack2', 'SeatBack3', 'SeatBack4', 'SeatBack5', 'SeatBack6']]

In [53]:
df["time_ard_mill"] = df["time_ard_mill"].round(0)
df["time"] = df["time_ard_mill"]

In [54]:
df.tail()

,date,time_ard_mill,Seat1,Seat2,Seat3,Seat4,Seat5,Seat6,SeatBack1,SeatBack2,SeatBack3,SeatBack4,SeatBack5,SeatBack6,time
78500,2017-12-20 20:03:15.594,-739835.0,0,0,0,82,0,0,0,0,0,0,0,0,-739835.0
78501,2017-12-20 20:03:15.695,-739734.0,0,0,0,84,0,0,0,0,0,0,0,0,-739734.0
78502,2017-12-20 20:03:15.794,-739635.0,0,0,0,82,0,0,0,0,0,0,0,0,-739635.0
78503,2017-12-20 20:03:15.894,-739535.0,0,0,0,84,0,0,0,0,0,0,0,0,-739535.0
78504,2017-12-20 20:03:15.994,-739436.0,0,0,0,81,0,0,0,0,0,0,0,0,-739436.0


In [55]:
df = df[['Seat1', 'Seat2', 'Seat3', 'Seat4', 'Seat5', 'Seat6', 'time', 'date']]

In [56]:
times = np.repeat(0, df.shape[0])
times[0] = 100

In [57]:
start = time.time()
for i in range(df.shape[0] - 1):
    times[i + 1] = times[i] + 100

print(time.time() - start)

0.05217766761779785


In [58]:
df['times'] = times

In [59]:
df.drop(columns = ['time'], inplace = True)

In [60]:
df.head()

,Seat1,Seat2,Seat3,Seat4,Seat5,Seat6,date,times
0,421,457,14,493,524,449,2017-12-20 17:52:25.737,100
1,423,457,14,493,524,444,2017-12-20 17:52:25.837,200
2,424,456,14,493,524,443,2017-12-20 17:52:25.937,300
3,424,456,14,492,524,443,2017-12-20 17:52:26.037,400
4,424,455,14,490,524,442,2017-12-20 17:52:26.137,500


In [61]:
df2 = df.copy()

### Now reading the eeg_ecg data

In [62]:
df_eeg_ecg2 = pd.read_csv("df_eeg_ecg2.csv")

In [63]:
df_eeg_ecg2.head()

,Unnamed: 0,Time,event,ibi,bpm,event.1,F3,F1,Fz,F2,F4,C3,C1,Cz,C2,C4,CPz,P3,P1,Pz,P2,P4,POz,O1,Oz,O2
0,0,1970-01-01 00:09:07.996000+00:00,1,1068.186279,56.169978,1,-2.8003,1.0988,-3.5002,-2.1005,-0.5952,-1.9005,-0.0010,-2.8003,-0.6008,-0.6181,1.4536,7.0974,8.7970,3.1983,0.4989,-3.1002,-3.0003,0.1990,-0.4009,-0.9249
1,1,1970-01-01 00:09:08.012000+00:00,2,961.875781,62.378117,2,-2.5196,1.4794,-2.8196,-1.5199,-0.7556,-1.5199,0.4797,-3.8193,-1.6198,-1.3284,0.0402,7.6780,8.3779,1.5794,-1.9198,-2.2197,-1.1200,1.6794,0.2797,-0.4323
2,2,1970-01-01 00:09:10.276000+00:00,1,976.511475,61.443210,1,-9.8154,-16.1139,-6.3162,-5.1164,-2.9864,4.7040,3.6815,-1.7172,1.6820,1.5259,3.2268,3.8270,6.4497,6.0810,-0.8174,2.7818,-0.5431,3.4816,3.0817,2.9031
3,3,1970-01-01 00:09:19.465000+00:00,1,923.660352,64.958943,1,-0.9183,-0.2184,0.7813,1.0813,0.7236,1.4812,0.4814,1.4812,-1.4416,-0.5408,0.3481,-0.3184,0.8813,0.0815,-1.2182,0.1913,-0.5235,-0.1184,-1.4181,-0.8165
4,4,1970-01-01 00:09:20.310000+00:00,2,853.735791,70.279354,2,-1.1854,1.7139,1.6140,2.7137,2.4692,-1.3853,0.3143,2.4138,1.3141,1.1321,0.4817,-2.3851,-0.8855,-0.9854,-0.4855,-1.4853,-1.3853,-0.9854,-1.6853,-1.3131


In [64]:
df_eeg_ecg2.drop(columns = ['Unnamed: 0'], inplace = True)

In [65]:
df_eeg_ecg2.head()

,Time,event,ibi,bpm,event.1,F3,F1,Fz,F2,F4,C3,C1,Cz,C2,C4,CPz,P3,P1,Pz,P2,P4,POz,O1,Oz,O2
0,1970-01-01 00:09:07.996000+00:00,1,1068.186279,56.169978,1,-2.8003,1.0988,-3.5002,-2.1005,-0.5952,-1.9005,-0.0010,-2.8003,-0.6008,-0.6181,1.4536,7.0974,8.7970,3.1983,0.4989,-3.1002,-3.0003,0.1990,-0.4009,-0.9249
1,1970-01-01 00:09:08.012000+00:00,2,961.875781,62.378117,2,-2.5196,1.4794,-2.8196,-1.5199,-0.7556,-1.5199,0.4797,-3.8193,-1.6198,-1.3284,0.0402,7.6780,8.3779,1.5794,-1.9198,-2.2197,-1.1200,1.6794,0.2797,-0.4323
2,1970-01-01 00:09:10.276000+00:00,1,976.511475,61.443210,1,-9.8154,-16.1139,-6.3162,-5.1164,-2.9864,4.7040,3.6815,-1.7172,1.6820,1.5259,3.2268,3.8270,6.4497,6.0810,-0.8174,2.7818,-0.5431,3.4816,3.0817,2.9031
3,1970-01-01 00:09:19.465000+00:00,1,923.660352,64.958943,1,-0.9183,-0.2184,0.7813,1.0813,0.7236,1.4812,0.4814,1.4812,-1.4416,-0.5408,0.3481,-0.3184,0.8813,0.0815,-1.2182,0.1913,-0.5235,-0.1184,-1.4181,-0.8165
4,1970-01-01 00:09:20.310000+00:00,2,853.735791,70.279354,2,-1.1854,1.7139,1.6140,2.7137,2.4692,-1.3853,0.3143,2.4138,1.3141,1.1321,0.4817,-2.3851,-0.8855,-0.9854,-0.4855,-1.4853,-1.3853,-0.9854,-1.6853,-1.3131


In [66]:
df2['Time'] = df2['date'] - pd.DateOffset(years = 47)
df2.head()

,Seat1,Seat2,Seat3,Seat4,Seat5,Seat6,date,times,Time
0,421,457,14,493,524,449,2017-12-20 17:52:25.737,100,1970-12-20 17:52:25.737
1,423,457,14,493,524,444,2017-12-20 17:52:25.837,200,1970-12-20 17:52:25.837
2,424,456,14,493,524,443,2017-12-20 17:52:25.937,300,1970-12-20 17:52:25.937
3,424,456,14,492,524,443,2017-12-20 17:52:26.037,400,1970-12-20 17:52:26.037
4,424,455,14,490,524,442,2017-12-20 17:52:26.137,500,1970-12-20 17:52:26.137


In [67]:
df2['Time'] = df2['Time'] - pd.DateOffset(hours = 8)
df2.head()

,Seat1,Seat2,Seat3,Seat4,Seat5,Seat6,date,times,Time
0,421,457,14,493,524,449,2017-12-20 17:52:25.737,100,1970-12-20 09:52:25.737
1,423,457,14,493,524,444,2017-12-20 17:52:25.837,200,1970-12-20 09:52:25.837
2,424,456,14,493,524,443,2017-12-20 17:52:25.937,300,1970-12-20 09:52:25.937
3,424,456,14,492,524,443,2017-12-20 17:52:26.037,400,1970-12-20 09:52:26.037
4,424,455,14,490,524,442,2017-12-20 17:52:26.137,500,1970-12-20 09:52:26.137


In [68]:
df2_micro = df2['date'].dt.microsecond

In [69]:
df2_micro

0        737000
1        837000
2        937000
3         37000
4        137000
          ...  
78500    594000
78501    695000
78502    794000
78503    894000
78504    994000
Name: date, Length: 78505, dtype: int64

In [70]:
df_eeg_ecg2_Time = pd.to_datetime(df_eeg_ecg2['Time'], format= '%Y-%m-%d %H:%M:%S.%f' )

In [71]:
df_eeg_ecg2_Time = df_eeg_ecg2_Time.dt.microsecond

In [72]:
df_eeg_ecg2_Time.head()

0    996000
1     12000
2    276000
3    465000
4    310000
Name: Time, dtype: int64

### Setting these extracted microseconds as new columns

In [73]:
df_eeg_ecg2['Time_millisecond'] = df_eeg_ecg2_Time/1000

In [74]:
df2['Time_millisecond'] = df2_micro/1000

In [75]:
df_to_model = pd.merge_asof(df_eeg_ecg2.sort_values('Time_millisecond'), df2.sort_values('Time_millisecond'), on = 'Time_millisecond')

# Trying to forecast based on EEG and ECG

In [76]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [77]:
df_to_model.head()

,Time_x,event,ibi,bpm,event.1,F3,F1,Fz,F2,F4,C3,C1,Cz,C2,C4,CPz,P3,P1,Pz,P2,P4,POz,O1,Oz,O2,Time_millisecond,Seat1,Seat2,Seat3,Seat4,Seat5,Seat6,date,times,Time_y
0,1970-01-01 00:16:06.001000+00:00,3,1074.287048,55.850994,3,0.0252,-1.0745,-0.9745,-2.3742,-1.7746,-0.6746,-2.6742,0.3252,-0.4747,-0.2827,0.2717,0.1252,0.3252,1.2250,1.0250,1.6249,0.9250,-0.5746,2.7246,2.2817,1.0,491,518,378,478,308,494,2017-12-20 18:25:44.001,1998400,1970-12-20 10:25:44.001
1,1970-01-01 00:17:34.003000+00:00,3,838.289600,71.574310,3,-0.5714,-2.0710,1.3282,-1.6711,-0.7395,2.3280,1.3282,-0.1715,1.9281,0.7260,0.8555,1.5281,-0.0715,1.6281,-0.6714,-0.8713,-0.9713,0.5284,-2.5709,-1.7977,3.0,472,543,425,501,307,528,2017-12-20 18:24:17.003,1911400,1970-12-20 10:24:17.003
2,1970-01-01 00:09:28.004000+00:00,2,838.285938,71.574623,2,-1.9363,0.2620,-1.0382,-1.8377,-0.5897,-0.5372,1.2600,0.5603,1.3609,0.8570,0.7790,-0.1387,-0.3372,0.4622,1.2625,-0.3391,-0.0372,0.5624,-0.6342,0.0591,4.0,0,0,0,0,0,0,2017-12-20 19:52:57.004,7231500,1970-12-20 11:52:57.004
3,1970-01-01 00:22:14.006000+00:00,1,937.483984,64.001093,1,-0.3519,0.5479,1.1477,1.0477,0.6384,0.1479,-0.4519,-0.5519,0.8478,0.2073,-0.2107,0.6478,-3.5512,1.2477,0.2479,-1.3517,-0.0520,0.3479,-0.0520,-0.5027,6.0,0,0,0,0,0,0,2017-12-20 19:53:17.006,7251500,1970-12-20 11:53:17.006
4,1970-01-01 00:09:08.012000+00:00,2,961.875781,62.378117,2,-2.5196,1.4794,-2.8196,-1.5199,-0.7556,-1.5199,0.4797,-3.8193,-1.6198,-1.3284,0.0402,7.6780,8.3779,1.5794,-1.9198,-2.2197,-1.1200,1.6794,0.2797,-0.4323,12.0,0,0,0,0,0,0,2017-12-20 19:47:47.012,6921500,1970-12-20 11:47:47.012


In [78]:
df_to_model.columns

Index(['Time_x', 'event', 'ibi', 'bpm', 'event.1', 'F3', 'F1', 'Fz', 'F2',
       'F4', 'C3', 'C1', 'Cz', 'C2', 'C4', 'CPz', 'P3', 'P1', 'Pz', 'P2', 'P4',
       'POz', 'O1', 'Oz', 'O2', 'Time_millisecond', 'Seat1', 'Seat2', 'Seat3',
       'Seat4', 'Seat5', 'Seat6', 'date', 'times', 'Time_y'],
      dtype='object')

In [79]:
y = df_to_model['event']

In [80]:
X = df_to_model.drop(columns = ['Time_x', 'Time_y', 'event.1', 'event', 'times', 'Time_millisecond', 'date'], axis = 1)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [82]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [83]:
clf.score(X_test, y_test)

0.5769230769230769